In [ ]:
#codigo_canhero = 6457
fecha_inicio = '2021-12-06'
fecha_final = '2021-12-11'

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime

import collections
from amigocloud import AmigoCloud

import collections

In [ ]:
acceso = open('../../accesos/api.txt','r')
api_token = acceso.readlines()[0]
acceso.close()

In [ ]:
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [ ]:
query_reporte = {'query': 'select canhero, propiedad, detalle, estado, gal.s3_filename, gal.amigo_id from dataset_272975 insp inner join dataset_276804 foto on insp.amigo_id = foto.fotografias inner join gallery_37796 gal on foto.amigo_id = gal.source_amigo_id where insp.fecha_registro between \'{fecha_ini}\' and \'{fecha_fin}\' order by canhero'.format(fecha_ini=fecha_inicio, fecha_fin=fecha_final)}
reporte_select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/30837/sql', query_reporte)
reporte_data = reporte_select['data']

#credito_data['insp_fech_inf'] = datetime.now().strftime('%d/%m/%Y')
#date_update = credito_data['date_update']
#credito_data['date_update'] = datetime.strptime(date_update[:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#credito = collections.namedtuple("credito", credito_data.keys())(*credito_data.values())

reporte_data


#print(query_reporte)
#print('end')

In [ ]:
import pandas as pd

In [ ]:
tabla = pd.DataFrame(reporte_data)

In [ ]:
tabla

In [ ]:
import requests
import docxtpl
from docxtpl import DocxTemplate
from docx.shared import Mm

In [ ]:
obs = reporte_data

In [ ]:
obs['canhero']

In [ ]:
doc = DocxTemplate("template/template02.docx")
lista_fotos_inline = []
for obs in reporte_data:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + obs['s3_filename'])
    file = open('fotos/' + obs['amigo_id']+'.jpg', "wb")
    file.write(response.content)
    file.close()
    print(obs['canhero'], obs['propiedad'])
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + obs['amigo_id']+'.jpg', width=Mm(70)), 'canhero':obs['canhero'], 'propiedad':obs['propiedad'], 'detalle': obs['detalle'], 'estado':obs['estado']})

data = collections.namedtuple("insp", obs.keys())(*obs.values())
context = {'insp':data, 'obs':lista_fotos_inline}
doc.render(context)
doc.save('_' + fecha_inicio + ' --  ' + fecha_final + '.docx')